In [1]:
# This Jupyter Notebook pairs a frame of image data with its ground truth counterpart (placed by the user in "quickstart-data"),
# and splits this pair of data into cubic volumes that are generated in the folder "quickstart-gendata"

# Import libraries

import numpy as np
import skimage
from skimage.util.shape import view_as_blocks
#import skimage.external.tifffile
import tifffile
import os
import shutil
import json
#from aicsimageio import AICSImage
import os 

In [10]:
# If you have a frame (and its ground truth) that you would wish to generate cubic data from, place them in the "quickstart-data" folder.
# Then replace the image_path and mask_path to point to these files.

# Change these paths for YOUR images.
image_path = r"/Users/dnadeem/Desktop/dl/pyLattice_deepLearning/src/quickstart-data/train_data_input.tif"
mask_path= r"/Users/dnadeem/Desktop/dl/pyLattice_deepLearning/src/quickstart-data/mask_all_frames_modified.tif"
split_directory="quickstart-gendata/"



# The side length of each cube (pixels). Will create patches with dimension (cube_size, cube_size, cube_size).
cube_size = 32
# Trying to get coverage of whole large dataset frame. Can change once we use more frames of our large data
train_split = 0.8 

# Create train and test folders in the split_directory.
if "train" not in os.listdir(split_directory):
    os.mkdir(split_directory+"train/")
if "test" not in os.listdir(split_directory):
    os.mkdir(split_directory+"test/")

In [3]:
# Read the image as an array
latticeMovieImage = tifffile.imread(image_path)
latticeMovieMask = tifffile.imread(mask_path)
#load tiff file to test 
# train_img = AICSImage(image_path)
# print(train_img.dims)
# latticeMovieImage = train_img.data
# latticeMovieImage = latticeMovieImage[:,0,:,:,:]
# labels_img = AICSImage(mask_path)
# print(labels_img.dims)
# latticeMovieMask = labels_img.data
# latticeMovieMask = latticeMovieMask[:,0,:,:,:]

print(latticeMovieImage.shape)
print(latticeMovieMask.shape)

(100, 55, 258, 275)
(100, 55, 258, 275)


In [4]:
np.amax(latticeMovieMask)

255.0

In [5]:
# If you need to invert the mask, use the line below
# latticeMovieMask = np.absolute(latticeMovieMask - 255.0)

frames = latticeMovieImage.shape[0]
latticeMovieImage_all = []
latticeMovieMask_all = []
frames = 5
for frame in range(frames):
    offset=np.asarray([0,0,0])
    
    x_extra = latticeMovieImage[frame].shape[2]%cube_size
    x_size = latticeMovieImage[frame].shape[2] - x_extra
    if offset[0] > x_extra:
        print("1st dim offset exceeds image dim")
        offset[0] = 0
        
    y_extra = latticeMovieImage[frame].shape[1]%cube_size
    y_size = latticeMovieImage[frame].shape[1] - y_extra
    if offset[1] > y_extra:
        print("2st dim offset exceeds image dim")
        offset[1] = 0
        
    z_extra = latticeMovieImage[frame].shape[0]%cube_size
    z_size = latticeMovieImage[frame].shape[0] - z_extra
    if offset[2] > z_extra:
        print("3rd dim offset exceeds image dim")
        offset[2] = 0
    
    # After calculating the extra pixels, we crop our frame so that the cubes can be perfectly divided.
    latticeMovieImage_frame = latticeMovieImage[frame,offset[0]:z_size+offset[0], offset[1]:y_size+offset[1], offset[2]:x_size+offset[2]]
    latticeMovieMask_frame = latticeMovieMask[frame,offset[0]:z_size+offset[0], offset[1]:y_size+offset[1], offset[2]:x_size+offset[2]]
    print("Image cropped to: " + str(z_size) + ", " + str(y_size) + ", " + str(x_size))
    latticeMovieImage_all.append(latticeMovieImage_frame)
    latticeMovieMask_all.append(latticeMovieMask_frame)
    
    
latticeMovieImage_all =  np.array(latticeMovieImage_all)
latticeMovieMask_all = np.array(latticeMovieMask_all)

print(latticeMovieImage_all.shape)
print(latticeMovieMask_all.shape)
print(np.amax(latticeMovieMask_all))

Image cropped to: 32, 256, 256
Image cropped to: 32, 256, 256
Image cropped to: 32, 256, 256
Image cropped to: 32, 256, 256
Image cropped to: 32, 256, 256
(5, 32, 256, 256)
(5, 32, 256, 256)
255.0


In [6]:
# Method to discard data with no or few targets in the ground truth
def filter_patches(lattice_patches, mask_patches, percent_covered=1e-10):
        zero_mask_ids = []
        
        for patch_index in range (0, mask_patches.shape[0]):
            patch = mask_patches[patch_index]
            if(np.count_nonzero(patch == 255.0)/(mask_patches.shape[1]**3) < percent_covered): #Means that the mask has all 0s
                zero_mask_ids.append(patch_index)
        
        lattice_patches = np.delete(lattice_patches, zero_mask_ids, axis=0)
        mask_patches = np.delete(mask_patches, zero_mask_ids, axis=0)
            
        return lattice_patches, mask_patches

In [7]:
lattice_patches = view_as_blocks(latticeMovieImage_all, block_shape=(1, cube_size, cube_size, cube_size))
lattice_patches.shape

(5, 1, 8, 8, 1, 32, 32, 32)

In [8]:
# Use the view_as_blocks and reshape methods to slice our frame into cubes.

lattice_patches = view_as_blocks(latticeMovieImage_all, block_shape=(1, cube_size, cube_size, cube_size))
lattice_patches = lattice_patches.reshape(int(frames)*int(x_size/cube_size)*int(y_size/cube_size)*int(z_size/cube_size), cube_size, cube_size, cube_size)

mask_patches = view_as_blocks(latticeMovieMask_all, block_shape=(1, cube_size, cube_size, cube_size))
mask_patches = mask_patches.reshape(int(frames)*int(x_size/cube_size)*int(y_size/cube_size)*int(z_size/cube_size), cube_size, cube_size, cube_size)

# Change percent_covered to set a threshold of what percent of pixels in the patch need to be signal (1).
lattice_patches, mask_patches = filter_patches(lattice_patches, mask_patches, percent_covered=1e-10)
print(lattice_patches.shape)
print(mask_patches.shape)

(210, 32, 32, 32)
(210, 32, 32, 32)


In [9]:
# Place the data ground-truth pair of cubes in their own folder within the quickstart-gendata directory

num_train_patches = int(train_split*lattice_patches.shape[0])

for k in range(0, num_train_patches):
    x_file = lattice_patches[k].astype('uint16')
    y_file = mask_patches[k].astype('uint16')
    
    metadata_x = dict(microscope='joh', shape=x_file.shape, dtype=x_file.dtype.str)
    metadata_x = json.dumps(metadata_x)
    
    metadata_y = dict(microscope='joh', shape=y_file.shape, dtype=y_file.dtype.str)
    metadata_y = json.dumps(metadata_y)
    
    os.mkdir(split_directory+"train/Region"+str(k)+"/")
    tifffile.imwrite(split_directory+"train/Region"+str(k)+"/"+"lattice_light_sheet.tif", x_file, description=metadata_x)
    tifffile.imwrite(split_directory+"train/Region"+str(k)+"/"+"truth.tif", y_file, description=metadata_y)
    
for k in range(num_train_patches, lattice_patches.shape[0]):
    x_file = lattice_patches[k].astype('uint16')
    y_file = mask_patches[k].astype('uint16')
    
    metadata_x = dict(microscope='joh', shape=x_file.shape, dtype=x_file.dtype.str)
    metadata_x = json.dumps(metadata_x)
    
    metadata_y = dict(microscope='joh', shape=y_file.shape, dtype=y_file.dtype.str)
    metadata_y = json.dumps(metadata_y)
    
    os.mkdir(split_directory+"test/Region"+str(k)+"/")
    tifffile.imwrite(split_directory+"test/Region"+str(k)+"/"+"lattice_light_sheet.tif", x_file, description=metadata_x)
    tifffile.imwrite(split_directory+"test/Region"+str(k)+"/"+"truth.tif", y_file, description=metadata_y)
    